# Notebook 2: Datenanalyse

**Projekt:** Analyse der Haushaltsausgaben in der Schweiz (2006-2022)  
**Autor:** CAS Information Engineering - ZHAW  
**Version:** 1.0  
**Datum:** 14. Oktober 2025

---

## Ziel dieses Notebooks

Dieses Notebook implementiert die **Datenanalyse** gemäß den funktionalen Anforderungen FA-02.1 bis FA-02.7:

- **FA-02.1:** Laden der bereinigten Daten
- **FA-02.2:** Analyse der Gesamtausgabenentwicklung
- **FA-02.3:** Analyse nach Altersgruppen
- **FA-02.4:** Analyse nach Haushaltstyp
- **FA-02.5:** Top-Wachstumskategorien identifizieren
- **FA-02.6:** Statistische Kennzahlen berechnen
- **FA-02.7:** Export der Analyseergebnisse

---

## Inhaltsverzeichnis

1. [Initialisierung und Laden der bereinigten Daten](#1-initialisierung)
2. [Gesamtausgabenentwicklung](#2-gesamtausgaben)
3. [Analyse nach Altersgruppen](#3-altersgruppen)
4. [Analyse nach Haushaltstyp](#4-haushaltstyp)
5. [Top-Wachstumskategorien](#5-wachstum)
6. [Statistische Kennzahlen](#6-statistik)
7. [Export der Analyseergebnisse](#7-export)
8. [Zusammenfassung](#8-zusammenfassung)

---

## 1. Initialisierung und Laden der bereinigten Daten <a id='1-initialisierung'></a>

### 1.1 Import der Bibliotheken

Wir importieren die notwendigen Bibliotheken für die Datenanalyse.

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Versionen anzeigen für Reproduzierbarkeit
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")

Pandas Version: 2.3.3
NumPy Version: 2.3.3


### 1.2 Pfade definieren

In [2]:
# Basis-Pfade definieren
BASE_DIR = Path.cwd().parent
PROCESSED_DATA_DIR = BASE_DIR / 'data' / 'processed'
OUTPUT_TABLES_DIR = BASE_DIR / 'outputs' / 'tables'

# Output-Ordner erstellen, falls nicht vorhanden
OUTPUT_TABLES_DIR.mkdir(parents=True, exist_ok=True)

print(f"Processed Data Directory: {PROCESSED_DATA_DIR}")
print(f"Output Tables Directory: {OUTPUT_TABLES_DIR}")
print(f"Processed-Ordner existiert: {PROCESSED_DATA_DIR.exists()}")
print(f"Output-Ordner existiert: {OUTPUT_TABLES_DIR.exists()}")

Processed Data Directory: c:\Users\samvo\source\repos\ScrpitingModulArbeit\data\processed
Output Tables Directory: c:\Users\samvo\source\repos\ScrpitingModulArbeit\outputs\tables
Processed-Ordner existiert: True
Output-Ordner existiert: True


### 1.3 Laden der bereinigten CSV-Dateien (FA-02.1)

In [3]:
def load_cleaned_data(filename, data_dir=PROCESSED_DATA_DIR):
    try:
        file_path = data_dir / filename
        df = pd.read_csv(file_path)
        print(f"Erfolgreich geladen: {filename}")
        print(f"  Shape: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"FEHLER: Datei nicht gefunden: {filename}")
        print(f"  Bitte führen Sie zuerst Notebook 01 aus!")
        return None
    except Exception as e:
        print(f"FEHLER beim Laden von {filename}: {e}")
        return None

In [4]:
print("LADEN DER BEREINIGTEN DATEN")
print("="*60)

df_gesamt = load_cleaned_data('clean_gesamtausgaben.csv')
df_alter = load_cleaned_data('clean_alter.csv')
df_haushaltstyp = load_cleaned_data('clean_haushaltstyp.csv')

print("="*60)
print("Laden abgeschlossen!")

LADEN DER BEREINIGTEN DATEN
Erfolgreich geladen: clean_gesamtausgaben.csv
  Shape: (1578, 4)
Erfolgreich geladen: clean_alter.csv
  Shape: (17927, 5)
Erfolgreich geladen: clean_haushaltstyp.csv
  Shape: (16135, 5)
Laden abgeschlossen!


### 1.4 Überprüfung der geladenen Daten

In [5]:
if df_gesamt is not None:
    print("GESAMTAUSGABEN - Überprüfung")
    df_gesamt.info()
    display(df_gesamt.head())

GESAMTAUSGABEN - Überprüfung
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578 entries, 0 to 1577
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   kategorie   1578 non-null   object 
 1   betrag_chf  1578 non-null   float64
 2   periode     1578 non-null   object 
 3   datentyp    1578 non-null   object 
dtypes: float64(1), object(3)
memory usage: 49.4+ KB


,kategorie,betrag_chf,periode,datentyp
0,50: Konsumausgaben,4948.529810,2022,Gesamt
1,51: Nahrungsmittel und alkoholfreie Getränke,629.206292,2022,Gesamt
2,511: Nahrungsmittel,577.607758,2022,Gesamt
3,5111: Brot und Getreideprodukte,99.190204,2022,Gesamt
4,5111.01: Reis,2.625455,2022,Gesamt


In [6]:
if df_alter is not None:
    print("AUSGABEN NACH ALTER - Überprüfung")
    df_alter.info()
    display(df_alter.head())
    if 'altersgruppe' in df_alter.columns:
        print("Altersgruppen:", df_alter['altersgruppe'].unique())

AUSGABEN NACH ALTER - Überprüfung
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17927 entries, 0 to 17926
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   kategorie     17927 non-null  object 
 1   betrag_chf    17927 non-null  float64
 2   periode       17927 non-null  object 
 3   altersgruppe  17927 non-null  object 
 4   datentyp      17927 non-null  object 
dtypes: float64(1), object(4)
memory usage: 700.4+ KB


,kategorie,betrag_chf,periode,altersgruppe,datentyp
0,50: Konsumausgaben,4657.166583,2020-2021,Total,Altersklasse
1,51: Nahrungsmittel und alkoholfreie Getränke,664.674230,2020-2021,Total,Altersklasse
2,511: Nahrungsmittel,608.632720,2020-2021,Total,Altersklasse
3,5111: Brot und Getreideprodukte,98.320600,2020-2021,Total,Altersklasse
4,5111.01: Reis,2.720182,2020-2021,Total,Altersklasse


Altersgruppen: ['Total' 'unter 30 Jahre' '30-49 Jahre' '50-64 Jahre'
 '65+ Jahre (Rentner)' 'übrige Nichterwerbstätige']


In [7]:
if df_haushaltstyp is not None:
    print("AUSGABEN NACH HAUSHALTSTYP - Überprüfung")
    df_haushaltstyp.info()
    display(df_haushaltstyp.head())
    if 'haushaltstyp' in df_haushaltstyp.columns:
        print("Haushaltstypen:", df_haushaltstyp['haushaltstyp'].unique())

AUSGABEN NACH HAUSHALTSTYP - Überprüfung
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16135 entries, 0 to 16134
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   kategorie     16135 non-null  object 
 1   betrag_chf    16135 non-null  float64
 2   periode       16135 non-null  object 
 3   haushaltstyp  16135 non-null  object 
 4   datentyp      16135 non-null  object 
dtypes: float64(1), object(4)
memory usage: 630.4+ KB


,kategorie,betrag_chf,periode,haushaltstyp,datentyp
0,50: Konsumausgaben,4657.166583,2020-2021,Total,Haushaltstyp
1,51: Nahrungsmittel und alkoholfreie Getränke,664.674230,2020-2021,Total,Haushaltstyp
2,511: Nahrungsmittel,608.632720,2020-2021,Total,Haushaltstyp
3,5111: Brot und Getreideprodukte,98.320600,2020-2021,Total,Haushaltstyp
4,5111.01: Reis,2.720182,2020-2021,Total,Haushaltstyp


Haushaltstypen: ['Total' 'Einpersonenhaushalt' 'Paarhaushalt ohne Kinder'
 'Paarhaushalt mit Kindern' 'Einelternhaushalt'
 'Anderer Mehrpersonenhaushalt']


---

## 2. Gesamtausgabenentwicklung (FA-02.2)

In [11]:
if df_gesamt is not None and 'periode' in df_gesamt.columns and 'betrag_chf' in df_gesamt.columns:
    ausgaben_pro_jahr = df_gesamt.groupby('periode')['betrag_chf'].sum().reset_index()
    ausgaben_pro_jahr.columns = ['jahr', 'gesamtausgaben_chf']
    ausgaben_pro_jahr['veraenderung_prozent'] = ausgaben_pro_jahr['gesamtausgaben_chf'].pct_change() * 100
    ausgaben_pro_jahr['veraenderung_absolut'] = ausgaben_pro_jahr['gesamtausgaben_chf'].diff()
    
    print("GESAMTAUSGABENENTWICKLUNG 2006-2022")
    print("="*80)
    display(ausgaben_pro_jahr)
    
    if len(ausgaben_pro_jahr) > 1:
        erste_jahr = ausgaben_pro_jahr.iloc[0]
        letzte_jahr = ausgaben_pro_jahr.iloc[-1]
        gesamtwachstum = ((letzte_jahr['gesamtausgaben_chf'] - erste_jahr['gesamtausgaben_chf']) / erste_jahr['gesamtausgaben_chf'] * 100)
        
        print(f"Ausgaben {int(erste_jahr['jahr'])}: CHF {erste_jahr['gesamtausgaben_chf']:,.2f}")
        print(f"Ausgaben {int(letzte_jahr['jahr'])}: CHF {letzte_jahr['gesamtausgaben_chf']:,.2f}")
        print(f"Gesamtwachstum: {gesamtwachstum:.2f}%")
        print(f"Durchschnittliches Wachstum: {ausgaben_pro_jahr['veraenderung_prozent'].mean():.2f}%")
else:
    print("Daten für Gesamtausgabenentwicklung nicht verfügbar")

GESAMTAUSGABENENTWICKLUNG 2006-2022


,jahr,gesamtausgaben_chf,veraenderung_prozent,veraenderung_absolut
0,2006,41787.159145,NaN,NaN
1,2015,43075.391683,3.082843,1288.232538
2,2022,44157.639384,2.512450,1082.247701


Ausgaben 2006: CHF 41,787.16
Ausgaben 2022: CHF 44,157.64
Gesamtwachstum: 5.67%
Durchschnittliches Wachstum: 2.80%


---

## 3. Analyse nach Altersgruppen (FA-02.3) <a id='3-altersgruppen'></a>

In [14]:
if df_alter is not None and 'altersgruppe' in df_alter.columns and 'betrag_chf' in df_alter.columns:
    ausgaben_nach_alter = df_alter.groupby('altersgruppe')['betrag_chf'].agg([('durchschnitt_chf', 'mean'), ('summe_chf', 'sum'), ('anzahl_eintraege', 'count')]).reset_index()
    ausgaben_nach_alter = ausgaben_nach_alter.sort_values('durchschnitt_chf', ascending=False)
    
    print("AUSGABEN NACH ALTERSGRUPPEN")
    print("="*80)
    display(ausgaben_nach_alter)
    
    hoechste_gruppe = ausgaben_nach_alter.iloc[0]
    niedrigste_gruppe = ausgaben_nach_alter.iloc[-1]
    
    print(f"Altersgruppe mit HÖCHSTEN Ausgaben: {hoechste_gruppe['altersgruppe']}")
    print(f"  Durchschnitt: CHF {hoechste_gruppe['durchschnitt_chf']:,.2f}")
    print(f"Altersgruppe mit NIEDRIGSTEN Ausgaben: {niedrigste_gruppe['altersgruppe']}")
    print(f"  Durchschnitt: CHF {niedrigste_gruppe['durchschnitt_chf']:,.2f}")
else:
    print("Daten für Altersgruppen-Analyse nicht verfügbar")

AUSGABEN NACH ALTERSGRUPPEN


,altersgruppe,durchschnitt_chf,summe_chf,anzahl_eintraege
1,50-64 Jahre,103.480207,321306.041996,3105
2,65+ Jahre (Rentner),98.994288,295794.933851,2988
0,30-49 Jahre,94.631259,289666.284356,3061
4,unter 30 Jahre,83.391287,232745.082515,2791
3,Total,83.358764,264413.998792,3172
5,übrige Nichterwerbstätige,76.278123,214341.526582,2810


Altersgruppe mit HÖCHSTEN Ausgaben: 50-64 Jahre
  Durchschnitt: CHF 103.48
Altersgruppe mit NIEDRIGSTEN Ausgaben: übrige Nichterwerbstätige
  Durchschnitt: CHF 76.28


---

## 4. Analyse nach Haushaltstyp (FA-02.4) <a id='4-haushaltstyp'></a>

In [15]:
if df_haushaltstyp is not None and 'haushaltstyp' in df_haushaltstyp.columns and 'betrag_chf' in df_haushaltstyp.columns:
    ausgaben_nach_haushalt = df_haushaltstyp.groupby('haushaltstyp')['betrag_chf'].agg([('durchschnitt_chf', 'mean'), ('summe_chf', 'sum'), ('anzahl_eintraege', 'count')]).reset_index()
    ausgaben_nach_haushalt = ausgaben_nach_haushalt.sort_values('durchschnitt_chf', ascending=False)
    
    print("AUSGABEN NACH HAUSHALTSTYP")
    print("="*80)
    display(ausgaben_nach_haushalt)
else:
    print("Daten für Haushaltstyp-Analyse nicht verfügbar")

AUSGABEN NACH HAUSHALTSTYP


,haushaltstyp,durchschnitt_chf,summe_chf,anzahl_eintraege
3,Paarhaushalt mit Kindern,109.959589,329768.807940,2999
0,Anderer Mehrpersonenhaushalt,108.527986,223567.650305,2060
1,Einelternhaushalt,88.301265,248656.362504,2816
5,Total,83.358764,264413.998792,3172
2,Einpersonenhaushalt,66.817021,190829.412029,2856
4,Paarhaushalt ohne Kinder,64.201133,143296.929847,2232


In [16]:
if df_haushaltstyp is not None and 'haushaltstyp' in df_haushaltstyp.columns and 'kategorie' in df_haushaltstyp.columns:
    haushaltstypen_unique = df_haushaltstyp['haushaltstyp'].unique()
    print("Verfügbare Haushaltstypen:", haushaltstypen_unique)
    
    familie_typen = [ht for ht in haushaltstypen_unique if 'familie' in str(ht).lower() or 'paar mit kind' in str(ht).lower()]
    single_typen = [ht for ht in haushaltstypen_unique if 'single' in str(ht).lower() or 'einpersonenhaushalt' in str(ht).lower() or '1 person' in str(ht).lower()]
    
    if familie_typen:
        df_familie = df_haushaltstyp[df_haushaltstyp['haushaltstyp'].isin(familie_typen)]
        top_kategorien_familie = df_familie.groupby('kategorie')['betrag_chf'].mean().sort_values(ascending=False).head(3)
        print("TOP 3 AUSGABENKATEGORIEN - FAMILIE")
        for idx, (kategorie, betrag) in enumerate(top_kategorien_familie.items(), 1):
            print(f"{idx}. {kategorie}: CHF {betrag:,.2f}")
    
    if single_typen:
        df_single = df_haushaltstyp[df_haushaltstyp['haushaltstyp'].isin(single_typen)]
        top_kategorien_single = df_single.groupby('kategorie')['betrag_chf'].mean().sort_values(ascending=False).head(3)
        print("TOP 3 AUSGABENKATEGORIEN - SINGLE")
        for idx, (kategorie, betrag) in enumerate(top_kategorien_single.items(), 1):
            print(f"{idx}. {kategorie}: CHF {betrag:,.2f}")

Verfügbare Haushaltstypen: ['Total' 'Einpersonenhaushalt' 'Paarhaushalt ohne Kinder'
 'Paarhaushalt mit Kindern' 'Einelternhaushalt'
 'Anderer Mehrpersonenhaushalt']
TOP 3 AUSGABENKATEGORIEN - SINGLE
1. 50: Konsumausgaben: CHF 3,701.83
2. 30: Obligatorische Transferausgaben: CHF 1,847.33
3. 57: Wohnen und Energie: CHF 1,281.64


---

## 5. Top-Wachstumskategorien (FA-02.5) <a id='5-wachstum'></a>

In [17]:
if df_gesamt is not None and 'periode' in df_gesamt.columns and 'kategorie' in df_gesamt.columns:
    
    # KORREKTUR: Finde das erste und letzte Jahr dynamisch
    start_jahr = df_gesamt['periode'].min()
    end_jahr = df_gesamt['periode'].max()

    print(f"Vergleiche Ausgaben zwischen {start_jahr} und {end_jahr}...")
    
    df_start = df_gesamt[df_gesamt['periode'] == start_jahr].groupby('kategorie')['betrag_chf'].mean()
    df_end = df_gesamt[df_gesamt['periode'] == end_jahr].groupby('kategorie')['betrag_chf'].mean()
    
    # DataFrame zusammenführen
    wachstum_kategorien = pd.DataFrame({f'mittelwert_{start_jahr}': df_start, f'mittelwert_{end_jahr}': df_end})
    
    # Veränderung berechnen und nach NaN filtern (falls eine Kategorie nur in einem Jahr existiert)
    wachstum_kategorien.dropna(inplace=True)
    wachstum_kategorien['veraenderung_prozent'] = ((wachstum_kategorien[f'mittelwert_{end_jahr}'] - wachstum_kategorien[f'mittelwert_{start_jahr}']) / wachstum_kategorien[f'mittelwert_{start_jahr}'] * 100)
    wachstum_kategorien = wachstum_kategorien.sort_values('veraenderung_prozent', ascending=False)
    
    print(f"\nTOP 5 WACHSTUMSKATEGORIEN {start_jahr}-{end_jahr}")
    print("="*80)
    top_5_wachstum = wachstum_kategorien.head(5)
    display(top_5_wachstum)
    
    for idx, (kategorie, row) in enumerate(top_5_wachstum.iterrows(), 1):
        print(f"{idx}. {kategorie}: {row['veraenderung_prozent']:.2f}% Wachstum")
else:
    print("Daten für Wachstumsanalyse nicht verfügbar")

Vergleiche Ausgaben zwischen 2006 und 2022...

TOP 5 WACHSTUMSKATEGORIEN 2006-2022


,mittelwert_2006,mittelwert_2022,veraenderung_prozent
kategorie,,,
4423.07: Gemachte Einladungen zu Mahlzeiten in Kantinen,0.581859,3.938703,576.917581
6632.07: Miete von Geräten und von DVDs und CDs,0.871433,4.494907,415.806502
4203.03: Prämien für die Reiseversicherung,0.638009,2.570778,302.937447
4423.08: Gemachte Einladungen zu Getränken in Kantinen,0.406761,1.506625,270.395505
4428.03: Andere verschenkte Waren und Dienstleistungen,8.191509,28.956318,253.491848


1. 4423.07: Gemachte Einladungen zu Mahlzeiten in Kantinen: 576.92% Wachstum
2. 6632.07: Miete von Geräten und von DVDs und CDs: 415.81% Wachstum
3. 4203.03: Prämien für die Reiseversicherung: 302.94% Wachstum
4. 4423.08: Gemachte Einladungen zu Getränken in Kantinen: 270.40% Wachstum
5. 4428.03: Andere verschenkte Waren und Dienstleistungen: 253.49% Wachstum


---

## 6. Statistische Kennzahlen (FA-02.6) <a id='6-statistik'></a>

In [20]:
if df_gesamt is not None and 'kategorie' in df_gesamt.columns:
    # Wir suchen nach einer Kategorie, die 'Wohnen' ENTHÄLT
    wohnkategorien = df_gesamt[df_gesamt['kategorie'].str.contains("Wohnen", na=False)]
    
    if not wohnkategorien.empty:
        # Wir nehmen die erste gefundene Kategorie für die Analyse
        interessante_kategorie = wohnkategorien['kategorie'].iloc[0]
        
        print(f"STATISTISCHE KENNZAHLEN - Analyse für die erste gefundene Wohn-Kategorie: '{interessante_kategorie}'")
        print("="*80)
        display(wohnkategorien['betrag_chf'].describe())
    else:
        print("Keine Kategorie mit dem Begriff 'Wohnen' in den Daten gefunden.")
else:
    print("Daten für statistische Analyse nicht verfügbar")

STATISTISCHE KENNZAHLEN - Analyse für die erste gefundene Wohn-Kategorie: '57: Wohnen und Energie'


count       3.000000
mean     1387.545316
std        27.443266
min      1369.793233
25%      1371.740891
50%      1373.688550
75%      1396.421357
max      1419.154164
Name: betrag_chf, dtype: float64

---

## 7. Export der Analyseergebnisse (FA-02.7) <a id='7-export'></a>

In [19]:
if 'wachstum_kategorien' in locals():
    wachstum_kategorien_export = wachstum_kategorien.reset_index()
    wachstum_kategorien_export.columns = ['kategorie', 'mittelwert_2006', 'mittelwert_2022', 'veraenderung_prozent']
    
    output_file = OUTPUT_TABLES_DIR / 'zusammenfassung.csv'
    wachstum_kategorien_export.to_csv(output_file, index=False, encoding='utf-8')
    
    print("EXPORT ERFOLGREICH")
    print("="*80)
    print(f"Datei gespeichert: {output_file}")
    print(f"Anzahl Zeilen: {len(wachstum_kategorien_export)}")
    
    print("Erste Zeilen der exportierten Daten:")
    display(wachstum_kategorien_export.head(10))
else:
    print("Keine Wachstumsdaten zum Export verfügbar")

EXPORT ERFOLGREICH
Datei gespeichert: c:\Users\samvo\source\repos\ScrpitingModulArbeit\outputs\tables\zusammenfassung.csv
Anzahl Zeilen: 504
Erste Zeilen der exportierten Daten:


,kategorie,mittelwert_2006,mittelwert_2022,veraenderung_prozent
0,4423.07: Gemachte Einladungen zu Mahlzeiten in...,0.581859,3.938703,576.917581
1,6632.07: Miete von Geräten und von DVDs und CDs,0.871433,4.494907,415.806502
2,4203.03: Prämien für die Reiseversicherung,0.638009,2.570778,302.937447
3,4423.08: Gemachte Einladungen zu Getränken in ...,0.406761,1.506625,270.395505
4,4428.03: Andere verschenkte Waren und Dienstle...,8.191509,28.956318,253.491848
5,"6700.01: Kindergarten, Primarschule, Real- und...",3.925167,12.055696,207.138390
6,5212.06: Weissweine ohne Herkunftsangabe,0.856291,2.478918,189.494989
7,"6831.00: Krippen, Spielgruppen und andere sozi...",15.470344,44.638619,188.543156
8,6831: Soziale Dienstleistungen,15.470344,44.638619,188.543156
9,6225.03: Billette für regionalen oder städtisc...,2.918483,7.624021,161.232296


---

## 8. Zusammenfassung <a id='8-zusammenfassung'></a>

### Durchgeführte Schritte

In diesem Notebook wurden folgende Aufgaben erfolgreich durchgeführt:

1. **FA-02.1:** Bereinigte Daten aus CSV-Dateien geladen
2. **FA-02.2:** Gesamtausgabenentwicklung 2006-2022 analysiert
3. **FA-02.3:** Ausgaben nach Altersgruppen analysiert
4. **FA-02.4:** Ausgaben nach Haushaltstyp verglichen
5. **FA-02.5:** Top-Wachstumskategorien identifiziert
6. **FA-02.6:** Statistische Kennzahlen berechnet
7. **FA-02.7:** Analyseergebnisse als CSV exportiert

### Ausgabedateien

Die folgende CSV-Datei wurde im Ordner `outputs/tables/` erstellt:

- `zusammenfassung.csv` - Wachstumsanalyse 2006-2022

### Nächste Schritte

Die Analyseergebnisse sind nun bereit für:

- **Notebook 3:** Visualisierung (FA-03)
- **Notebook 4:** Business Case Dokumentation (FA-04)